## 说明

这是用于检验torch计算期权价格和深度学习定价的文件, 可以自由调取相应的计算脚本, 使用cpu和gpu以及不同的框架对传统的显示推演法以及深度学习定价法进行速度和精度的检验

注意, 本框架暂时不具备实际应用价值, 后续会开发cython和cupyx的加速版本

In [1]:
from option_calc import OptionPrc
from utils import params
from test_func import *

#### torch.tensor加速BS定价

In [2]:
cpu_runner = OptionPrc(params)

#### tensor_backwardation和bs公式显式计算的结果对比

In [3]:
greeks = greeks_stats(methods={cpu_runner.bs_greeks.__name__: cpu_runner.bs_greeks(),
                               cpu_runner.autograd_greeks.__name__: cpu_runner.autograd_greeks()})
greeks

,Delta,Gamma,Vega,Theta,Rho
bs_greeks,0.046362,0.019417,9.708576,0.288136,4.542206
autograd_greeks,0.046361,0.019417,9.708447,0.288132,4.542130


#### 计算速度对比

In [4]:
calc_time = time_stats(methods={cpu_runner.bs_greeks.__name__: cpu_runner.bs_greeks
                               , cpu_runner.autograd_greeks.__name__: cpu_runner.autograd_greeks}
                               , num_runs=1000, num_repeats=5)
calc_time

,Avg Time per Call (μs)
bs_greeks,1086.36746
autograd_greeks,1473.31440


可以看到用cpu来求导会比显式求greeks要慢一些

#### GPU计算速度对比

In [5]:
gpu_runner = OptionPrc(params, device='gpu')
greeks = greeks_stats(methods={gpu_runner.bs_greeks.__name__: gpu_runner.bs_greeks(),
                               gpu_runner.autograd_greeks.__name__: gpu_runner.autograd_greeks()})
greeks

,Delta,Gamma,Vega,Theta,Rho
bs_greeks,0.046362,0.019417,9.708576,0.288136,4.542206
autograd_greeks,0.046361,0.019417,9.708447,0.288132,4.542130


In [6]:
calc_time = time_stats(methods={cpu_runner.bs_greeks.__name__: cpu_runner.bs_greeks
                               , cpu_runner.autograd_greeks.__name__: cpu_runner.autograd_greeks
                               , gpu_runner.bs_greeks.__name__: gpu_runner.bs_greeks
                               , gpu_runner.autograd_greeks.__name__: gpu_runner.autograd_greeks}
                               , num_runs=1000, num_repeats=5)
calc_time

,Avg Time per Call (μs)
bs_greeks,1110.79210
autograd_greeks,1455.20468


最简单的欧式期权下torch框架并没有表现出优势